In [1]:
import pandas as pd
import numpy as np
from itertools import product

In [2]:
df = pd.read_csv(
    "data/6.txt", sep=r"\s*", engine="python", names=range(-1, 1000)
).dropna(how="all", axis=1)
symbols = {"^": (-1, 0), ">": (0, 1), "v": (1, 0), "<": (0, -1)}
cycle = list(symbols.keys())
row_indices, col_indices = np.where(df.isin(symbols.keys()))

In [3]:
x, y = row_indices[0], col_indices[0]
d = df.values[x, y]
dx, dy = symbols[d]
visits = df.copy()

while 0 <= x + dx < len(df) and 0 <= y + dy < len(df.columns):
    visits.values[x, y] = "X"
    while df.values[x + dx, y + dy] == "#":
        d = cycle[(cycle.index(d) + 1) % len(cycle)]
        dx, dy = symbols[d]
    x, y = x + dx, y + dy
visits.values[x, y] = "X"

(visits == "X").values.sum()

np.int64(4967)

In [4]:
def detect_loop(start_x, start_y, obstacle_x, obstacle_y):
    d, x, y = df.values[start_x, start_y], start_x, start_y
    dx, dy = symbols[d]
    restore = df.values[obstacle_x, obstacle_y]
    df.values[obstacle_x, obstacle_y] = "#"
    visited = set()

    try:
        while 0 <= x + dx < len(df) and 0 <= y + dy < len(df.columns):
            if (x, y, d) in visited:
                return True
            visited.add((x, y, d))
            c = 0
            while df.values[x + dx, y + dy] == "#":
                c += 1
                d = cycle[(cycle.index(d) + 1) % len(cycle)]
                dx, dy = symbols[d]
                if c > 4:
                    return False
            x, y = x + dx, y + dy
        return False
    finally:
        df.values[obstacle_x, obstacle_y] = restore


row_indices, col_indices = np.where(df.isin(symbols.keys()))
start_x, start_y = row_indices[0], col_indices[0]

res = df.copy()
for i, j in product(range(len(df)), range(len(df.columns))):
    if df.values[i, j] != ".":
        continue
    if visits.values[i, j] != "X":
        continue
    if detect_loop(start_x, start_y, i, j):
        res.values[i, j] = "O"

(res == "O").values.sum()

np.int64(1789)